In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from PIL import Image
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [8]:
# --- Предыдущие настройки ---
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[idx, 1])
        image = Image.open(img_path).convert("RGB")
        label = int(self.annotations.iloc[idx, 2])
        
        if self.transform:
            image = self.transform(image)

        return image, label

In [9]:
# Пути к CSV-файлам и корневая папка
train_csv = '/home/andrey/Downloads/picture_classifier/data/EuroSAT/train.csv'
valid_csv = '/home/andrey/Downloads/picture_classifier/data/EuroSAT/validation.csv'
image_root_dir = '/home/andrey/Downloads/picture_classifier/data/EuroSAT'

In [10]:
# Применяем трансформации
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

valid_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [11]:
# Создаем кастомный датасет и DataLoader
train_dataset = CustomDataset(csv_file=train_csv, root_dir=image_root_dir, transform=train_transforms)
valid_dataset = CustomDataset(csv_file=valid_csv, root_dir=image_root_dir, transform=valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [12]:
# Проверка работы DataLoader
data_iter = iter(train_loader)
images, labels = next(data_iter)
print(f"Размер загруженных изображений: {images.shape}")
print(f"Размер загруженных меток: {labels.shape}")

# Устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используемое устройство: {device}")

# --- Настройка моделей ---
# Количество классов
num_classes = len(train_dataset.annotations['Label'].unique())
print(f"Количество классов: {num_classes}")

Размер загруженных изображений: torch.Size([32, 3, 224, 224])
Размер загруженных меток: torch.Size([32])
Используемое устройство: cuda
Количество классов: 10


In [13]:
# Модель 1: ResNet-50
resnet_model = models.resnet50(pretrained=True)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, num_classes)
resnet_model = resnet_model.to(device)

# Модель 2: EfficientNet-B0
efficientnet_model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
efficientnet_model.classifier[1] = nn.Linear(efficientnet_model.classifier[1].in_features, num_classes)
efficientnet_model = efficientnet_model.to(device)

/home/andrey/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/andrey/miniforge3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /home/andrey/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 83.2MB/s]


In [14]:
# Замораживаем все параметры, кроме последнего слоя
for param in resnet_model.parameters():
    param.requires_grad = False
for param in resnet_model.fc.parameters():
    param.requires_grad = True

for param in efficientnet_model.parameters():
    param.requires_grad = False
for param in efficientnet_model.classifier[1].parameters():
    param.requires_grad = True

In [15]:
# Определяем функцию потерь и оптимизаторы для каждой модели
criterion = nn.CrossEntropyLoss()
optimizer_resnet = optim.Adam(resnet_model.fc.parameters(), lr=0.001)
optimizer_efficientnet = optim.Adam(efficientnet_model.classifier[1].parameters(), lr=0.001)

In [16]:
# --- Функция для обучения ---
def train_model(model, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Обнуляем градиенты
            optimizer.zero_grad()

            # Прямой проход
            outputs = model(images)

            # Потери и оптимизация
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Эпоха [{epoch+1}/{num_epochs}], Потеря: {running_loss/len(train_loader):.4f}")

In [17]:
# --- Функция для оценки ---
def evaluate_model(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [18]:
# --- Обучение и оценка ResNet-50 ---
print("\nОбучение модели ResNet-50")
train_model(resnet_model, optimizer_resnet, num_epochs=10)
resnet_accuracy = evaluate_model(resnet_model)
print(f'Точность ResNet-50: {resnet_accuracy:.2f}%')


Обучение модели ResNet-50
Эпоха [1/10], Потеря: 0.5003
Эпоха [2/10], Потеря: 0.2928
Эпоха [3/10], Потеря: 0.2633
Эпоха [4/10], Потеря: 0.2412
Эпоха [5/10], Потеря: 0.2377
Эпоха [6/10], Потеря: 0.2319
Эпоха [7/10], Потеря: 0.2050
Эпоха [8/10], Потеря: 0.2093
Эпоха [9/10], Потеря: 0.2082
Эпоха [10/10], Потеря: 0.1973
Точность ResNet-50: 93.94%


In [19]:
# --- Обучение и оценка EfficientNet-B0 ---
print("\nОбучение модели EfficientNet-B0")
train_model(efficientnet_model, optimizer_efficientnet, num_epochs=10)
efficientnet_accuracy = evaluate_model(efficientnet_model)
print(f'Точность EfficientNet-B0: {efficientnet_accuracy:.2f}%')


Обучение модели EfficientNet-B0
Эпоха [1/10], Потеря: 0.5964
Эпоха [2/10], Потеря: 0.3675
Эпоха [3/10], Потеря: 0.3323
Эпоха [4/10], Потеря: 0.3196
Эпоха [5/10], Потеря: 0.2994
Эпоха [6/10], Потеря: 0.3003
Эпоха [7/10], Потеря: 0.2884
Эпоха [8/10], Потеря: 0.2914
Эпоха [9/10], Потеря: 0.2831
Эпоха [10/10], Потеря: 0.2851
Точность EfficientNet-B0: 93.02%


In [20]:
# --- Сравнение результатов ---
print("\nСравнение моделей:")
print(f"ResNet-50 точность: {resnet_accuracy:.2f}%")
print(f"EfficientNet-B0 точность: {efficientnet_accuracy:.2f}%")


Сравнение моделей:
ResNet-50 точность: 93.94%
EfficientNet-B0 точность: 93.02%


In [23]:
# Сохранение ResNet-50
torch.save(resnet_model.state_dict(), '/home/andrey/Downloads/picture_classifier/models/resnet50_eurosat.pth')
print("ResNet-50 сохранена в '/home/andrey/Downloads/picture_classifier/models/resnet50_eurosat.pth'")

# Сохранение EfficientNet-B0
torch.save(efficientnet_model.state_dict(), '/home/andrey/Downloads/picture_classifier/models/efficientnet_b0_eurosat.pth')
print("EfficientNet-B0 сохранена в '/home/andrey/Downloads/picture_classifier/models/efficientnet_b0_eurosat.pth'")

ResNet-50 сохранена в '/home/andrey/Downloads/picture_classifier/models/resnet50_eurosat.pth'
EfficientNet-B0 сохранена в '/home/andrey/Downloads/picture_classifier/models/efficientnet_b0_eurosat.pth'
